### 아마존 리뷰 크롤링

#### 필수 라이브러리 불러오기

In [1]:
# 필수라이브러리 설치 

from selenium import webdriver           # 동적 크롤링을 가능하게 하는 selenium. 셀레니움에서 webdriver를 끌고 온다. why? webdriver가 매크로를 끌고 오는 것 이기 때문이다. 
import time                              # time 이라는 모듈을 끌고 온다. 이것을 끌고 오지 않으면 반복되는 요청에 과부화(디도스 공격)라고 생각해 다운되기 때문이다. 
from bs4 import BeautifulSoup as bs     # HTML정보로 부터 원하는 데이터를 가져오기 쉽게, 비슷한 분류의 데이터별로 나누어주는(parsing) 파이썬 라이브러리.
import csv                               # csv 모듈을 끌고온다. csv 모듈은 CSV 형식의 표 형식 데이터를 읽고 쓰는 클래스를 구현
import requests                          # requests 모듈은 HTTP Request를 웹 브라우저가 아닌 python에서 모듈로서 가능하게 해주는 모듈
import pandas as pd                      # 파이썬에서 데이터 프레임으로 다루기 위해 필요한 모듈.
import re                                # 정규표현을 위한 모듈
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities # Desired Capabilities Class는 테스트 스크립트에서 사용할 환경을 웹 드라이버에게 알려주는 데 필요함.
from selenium.webdriver.chrome.options import Options # Options class 는 크롬 옵션을 설정하기 위하여 필요한 클라스
import warnings 
warnings.filterwarnings(action='ignore') #경고 메세지 안나오게 함

#### 크롬 드라이버 다운로드 방법

- 1. https://sites.google.com/chromium.org/driver/ 접속
- 2. All versions available in Downloads 에 Latest stable release: 
- 3. 각 노트북 및 컴퓨터에 맞는 크롬 드라이버 다운로드

#### 크롤링에 필요한 옵션 설정 및 리스트 생성

In [2]:
# 각 크롤링 대상 리스트 생성 및 크롬 드라이버 

date_list = [] # 날짜
star_list = [] # 별점
review_list = [] # 리뷰 내용
helpful_list = [] # 도움이 되었어요.

options = webdriver.ChromeOptions() # 크롬 옵션을 가져오고 선언
options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'}) # 크롬 언어 옵션 수정 : 모국어 페이지를 영어로 번역.

# 크롬 드라이버 위치 설정
driver = webdriver.Chrome("C:\\Users\\mh\\jupyter code\\chromedriver.exe", chrome_options = options)

#### 본격적인 크롤링!

- 스마트 TV : LG 80 Series 65” Alexa Built-in, 4K UHD Smart TV, Native 60Hz Refresh Rate, Dolby Cinema, Director Settings, Gaming Mode, with Magic Remote (65UP8000, 2021)

In [3]:
for i in range(1,5):
    print(i)

1
2
3
4


In [4]:
for i in range(1,5): #더 많은 리뷰를 수집하고자 하면 range를 변경, 1페이지에 리뷰 10개씩

    driver.implicitly_wait(1)  #implicitly wait(웹페이지가 넘어올때까지 기다리기), explicitly wait(웹페이지 원하는 부분이 나타날때까지 기다리기)

    #아마존 리뷰 패턴이 url 마지막에 페이지 넘버 format 변경
    url = 'https://www.amazon.com/LG-65UP8000PUA-Alexa-Built-Smart/product-reviews/B08WHRW6QR/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={0}'.format(i)

    driver.get(url) # 해당 URL을 브라우저에서 띄움.
    time.sleep(1) # time 라이브러리의 sleep 함수를 사용하여 일정 시간동안 프로세스를 일시정지.

    soup = bs(driver.page_source, 'lxml') #webdriver가 연 url의 html은 page_source를 통해 가져옴
        
    # 날짜 크롤링
    dates = soup.find_all('span', {'data-hook':'review-date'})
    for date in dates:
        date_list.append(date.text.split("on")[1])
               
    # 별점 크롤링 
    stars1 = soup.find_all('i',{'data-hook':'review-star-rating'}) 
    for star in stars1 :
        star_list.append(float(star.text.split(" ")[0]))
        
    #별점 형태가 2개
    #stars2 = soup.find_all('i',{'data-hook':'cmps-review-star-rating'})
    #for star in stars2 :
        #star_list.append(float(star.text.split(" ")[0]))    
    
    # 도움이 된 글, 공백발생
    helpfuls = soup.find_all('span', class_='a-size-base a-color-tertiary cr-vote-text')      
    for i in range(len(dates)):
        try:
            helpful_list.append((helpfuls[i].text.split(" ")[0].replace(',',"")))
        except:
            helpful_list.append(0)   
            
    # 리뷰 내용         
    reviews = soup.find_all('span', class_= "a-size-base review-text review-text-content")
    #reviews = soup.find_all('span', {"class":{"a-size-base review-text review-text-content","a-row a-spacing-small review-data"}})
                             
    for review in reviews:
        review_list.append(review.text)

In [5]:
# 수집된 데이터 개수 확인
len(date_list), len(star_list), len(review_list), len(helpful_list)

(40, 40, 40, 40)

In [6]:
# 수집한 리스트를 DataFrame으로 전환
data = {'date' : date_list,
        'star' : star_list,
        'review' : review_list,
        'helpful': helpful_list}
df = pd.DataFrame(data)
df.head(15)

,date,star,review,helpful
0,"April 28, 2021",3.0,\nFirst tv arrived with a shattered screen. Am...,126
1,"April 30, 2021",4.0,\nThis TV is excellent. Color is good compares...,101
2,"May 28, 2021",1.0,"\nReceived my new TV. No damage, screen is pe...",76
3,"April 17, 2021",2.0,\nOur TV arrived completely shattered. We've s...,48
4,"June 6, 2021",5.0,\nFor a $600 tv i love it. I dont know why peo...,33
5,"April 24, 2021",5.0,\nArrived in perfect shape and easy to set up ...,43
6,"May 7, 2021",5.0,\nWhat a great buy! so happy with this. Very l...,37
7,"May 11, 2021",4.0,\nTHE MANUAL IS NOT GOOD ENOUGH TO EXPLAIN THE...,35
8,"June 3, 2021",1.0,\nSpent hours trying to get this to work. For...,25
9,"June 27, 2021",1.0,\nThis TV worked for about a month. Now nothin...,22


In [7]:
#helpful에서 One으로 돼있는 행 처리
for i in range(len(df)):
    if df.helpful.iloc[i] == 'One':
        df.helpful.iloc[i] = 1

In [8]:
df.head(15)

,date,star,review,helpful
0,"April 28, 2021",3.0,\nFirst tv arrived with a shattered screen. Am...,126
1,"April 30, 2021",4.0,\nThis TV is excellent. Color is good compares...,101
2,"May 28, 2021",1.0,"\nReceived my new TV. No damage, screen is pe...",76
3,"April 17, 2021",2.0,\nOur TV arrived completely shattered. We've s...,48
4,"June 6, 2021",5.0,\nFor a $600 tv i love it. I dont know why peo...,33
5,"April 24, 2021",5.0,\nArrived in perfect shape and easy to set up ...,43
6,"May 7, 2021",5.0,\nWhat a great buy! so happy with this. Very l...,37
7,"May 11, 2021",4.0,\nTHE MANUAL IS NOT GOOD ENOUGH TO EXPLAIN THE...,35
8,"June 3, 2021",1.0,\nSpent hours trying to get this to work. For...,25
9,"June 27, 2021",1.0,\nThis TV worked for about a month. Now nothin...,22


In [9]:
df.to_csv("아마존 스마트티비 교육용.csv", encoding='utf-8') 